# Import

In [25]:
from duckdb import connect as duck_connect
import psycopg2

# Load data from the redisDB

In [26]:
path = "/Users/julien/Downloads/hn.duckdb"

con = duck_connect(path, read_only=True)

"""
Example:
[(8818870,
  'Show HN: Intentionally subtle headphone crossfeed filter',
  'https://github.com/LnxPrgr3/crossfeed',
  51,
  1420022572,
  21,
  'LnxPrgr3')]

Schema:
CREATE TABLE hn_post (
    id          INTEGER PRIMARY KEY,
    type        TEXT NOT NULL,
    by          TEXT,
    time        BIGINT, -- Unix epoch time
    text        TEXT,
    parent      INTEGER,
    kids        TEXT[],
    url         TEXT,
    descendants INTEGER,
    score       INTEGER,
    title       TEXT,
    deleted     BOOLEAN,
    dead        BOOLEAN
);

Schema story:
id:int,
title:string,
url:string,
score:int,
time:int,
comments:int,
author:string
"""

print("Done loading data")


Done loading data


In [27]:
# Postgres
from time import sleep
conP = psycopg2.connect("dbname=hn user=postgres host=localhost port=5432")
curP = conP.cursor()

# Get the count of rows in duckdb
con.execute("SELECT COUNT(*) FROM story;")
count = con.fetchone()[0]

for i in range(0, count, 1000):
    res = con.execute(
        f"SELECT * FROM story ORDER BY time DESC LIMIT 1000 OFFSET {i};").fetchdf()
    for row in res.itertuples():
        try:
            curP.execute(
                "INSERT INTO hn_post VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, FALSE, FALSE);",
                (row.id,
                "post",
                row.author,
                row.time,
                None,
                None,
                None,
                row.url if row.url else None,
                row.comments,
                row.score,
                row.title,
                ))
            
        except Exception as e:
            continue
            

    conP.commit()
    print(f"\r{i}/{count}", end="")


4021000/4021340

In [28]:
# Count = 4021340

# Add the last 340 rows
res = con.execute(
    f"SELECT * FROM story ORDER BY time DESC LIMIT 1000 OFFSET 4021000;").fetchdf()
for row in res.itertuples():
    try:
        curP.execute(
            "INSERT INTO hn_post VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);",
            (row.id,
            "post",
            row.author,
            row.time,
            None,
            None,
            None,
            row.url,
            row.comments,
            row.score,
            row.title,
            ))
        
    except Exception as e:
        continue

conP.commit()

# Embedding transfer
Path: `../story.csv`

**Schema csv:**
id:int,
title:string,
url:string,
score:int,
time:int,
comments:int,
author:string,
embeddings:string

```SQL
CREATE TABLE hn_embeddings (
    id INTEGER PRIMARY KEY references hn_post,
    embedding vector(1536)
);
```

We want to transfer the embeddings from the csv to the database table `hn_embeddings`.



In [42]:
import pandas as pd

df = pd.read_csv("../story.csv")


In [48]:
# Abort failed transactions
curP.execute("ROLLBACK;")

In [51]:
# Transfer the data
i = 0
for row in df.iterrows():
    if i % 100 == 0:
        print(f"\r{i}/{len(df)}", end="")

    i += 1
    try:
        curP.execute(
            "INSERT INTO hn_embeddings VALUES (%s, %s);",
            (row[1].id,
             row[1].embeddings,
             ))
        conP.commit()
    except Exception as e:
        curP.execute("ROLLBACK;")
        

conP.commit()
        

108400/108477